## Tic Tac Toe environment

In [5]:
import itertools
import random
import numpy as np

class TicTacToeEnv:
    def __init__(self):
        # Pre-compute all possible states more efficiently
        self.states = list(itertools.product(["X", "O", " "], repeat=9))
        self.win_positions = [
            (0, 1, 2), (3, 4, 5), (6, 7, 8),  # Rows
            (0, 3, 6), (1, 4, 7), (2, 5, 8),  # Columns
            (0, 4, 8), (2, 4, 6)  # Diagonals
        ]

    @staticmethod
    def actions(state):
        return [i for i, s in enumerate(state) if s == " "]

    @staticmethod
    def transition_model(state, action, player):
        return tuple(player if i == action else s for i, s in enumerate(state))

    def reward(self, state, player):
        # More comprehensive reward function
        opponent = "O" if player == "X" else "X"

        # Check for win
        for pos in self.win_positions:
            if state[pos[0]] == state[pos[1]] == state[pos[2]] == player:
                return 10  # Higher reward for winning

        # Check for draw
        if " " not in state:
            return 0

        # Strategic position rewards
        reward = 0

        # Center control (high strategic value)
        if state[4] == player:
            reward += 2

        # Corners control
        corner_positions = [0, 2, 6, 8]
        corner_count = sum(1 for i in corner_positions if state[i] == player)
        reward += corner_count

        # Block opponent's potential win
        for pos in self.win_positions:
            if sum(1 for i in pos if state[i] == opponent) == 2 and \
               sum(1 for i in pos if state[i] == " ") == 1:
                reward += 3  # High reward for blocking potential win

        # Potential win setup
        for pos in self.win_positions:
            if sum(1 for i in pos if state[i] == player) == 2 and \
               sum(1 for i in pos if state[i] == " ") == 1:
                reward += 4  # Even higher reward for setting up a win

        return reward - 5  # Baseline penalty to encourage decisive moves

    @staticmethod
    def is_terminal(state):
        # Simplified terminal state check
        win_positions = [(0, 1, 2), (3, 4, 5), (6, 7, 8), (0, 3, 6), (1, 4, 7), (2, 5, 8), (0, 4, 8), (2, 4, 6)]
        return any(state[pos[0]] == state[pos[1]] == state[pos[2]] != " " for pos in win_positions) or " " not in state


## Value Iteration

In [ ]:
class ValueIteration:
    def __init__(self, theta=0.001, discount_factor=0.9):
        self.env = TicTacToeEnv()
        self.theta = theta
        self.discount_factor = discount_factor
        self.states = self.env.states
        self.V = {s: 0 for s in self.states}
        self.policy = {}

    def value_iteration(self):
        iterations = 0
        max_iterations = 1000  # Prevent infinite loops

        while iterations < max_iterations:
            delta = 0
            for s in self.states:
                if self.env.is_terminal(s):
                    self.V[s] = 0
                    continue

                old_value = self.V[s]
                
                # Compute best action value
                action_values = []
                for a in self.env.actions(s):
                    next_state = self.env.transition_model(s, a, "X")
                    reward = self.env.reward(next_state, "X")
                    action_value = reward + self.discount_factor * self.V.get(next_state, 0)
                    action_values.append(action_value)

                if action_values:
                    self.V[s] = max(action_values)
                    self.policy[s] = self.env.actions(s)[np.argmax(action_values)]

                delta = max(delta, abs(old_value - self.V[s]))

            if delta < self.theta:
                break
            
            iterations += 1

    def get_best_action(self, state):
        return self.policy.get(state, random.choice(self.env.actions(state)))

    @staticmethod
    def print_board(state):
        for i in range(0, 9, 3):
            print(state[i:i+3])
        print("\n")

def play_game():
    game = ValueIteration()
    game.value_iteration()

    state = (" ",) * 9
    current_player = "X"

    while not TicTacToeEnv.is_terminal(state):
        game.print_board(state)

        if current_player == "X":
            while True:
                try:
                    action = int(input("Enter your move (0-8): "))
                    if action in TicTacToeEnv.actions(state):
                        break
                    else:
                        print("Invalid move. Try again.")
                except ValueError:
                    print("Please enter a number between 0-8.")

        else:
            action = game.get_best_action(state)

        state = TicTacToeEnv.transition_model(state, action, current_player)
        current_player = "O" if current_player == "X" else "X"

    game.print_board(state)

    # Determine winner
    for player in ["X", "O"]:
        if any(state[pos[0]] == state[pos[1]] == state[pos[2]] == player for pos in game.env.win_positions):
            print(f"Player {player} wins!")
            return

    print("It's a draw!")

if __name__ == "__main__":
    play_game()

(' ', ' ', ' ')
(' ', ' ', ' ')
(' ', ' ', ' ')


(' ', ' ', ' ')
(' ', 'X', ' ')
(' ', ' ', ' ')


('O', ' ', ' ')
(' ', 'X', ' ')
(' ', ' ', ' ')


('O', ' ', ' ')
(' ', 'X', ' ')
(' ', ' ', 'X')


('O', ' ', 'O')
(' ', 'X', ' ')
(' ', ' ', 'X')


('O', 'X', 'O')
(' ', 'X', ' ')
(' ', ' ', 'X')


('O', 'X', 'O')
(' ', 'X', ' ')
('O', ' ', 'X')


('O', 'X', 'O')
(' ', 'X', 'X')
('O', ' ', 'X')


('O', 'X', 'O')
('O', 'X', 'X')
('O', ' ', 'X')


Player O wins!


## Policy iteration

In [7]:
import itertools
import random
import numpy as np

class TicTacToeEnv:
    def __init__(self):
        self.states = list(itertools.product(["X", "O", " "], repeat=9))
        self.win_positions = [
            (0, 1, 2), (3, 4, 5), (6, 7, 8),  # Rows
            (0, 3, 6), (1, 4, 7), (2, 5, 8),  # Columns
            (0, 4, 8), (2, 4, 6)  # Diagonals
        ]

    @staticmethod
    def actions(state):
        return [i for i, s in enumerate(state) if s == " "]

    @staticmethod
    def transition_model(state, action, player):
        return tuple(player if i == action else s for i, s in enumerate(state))

    def evaluate_state(self, state, player):
        opponent = "O" if player == "X" else "X"
        
        # Check for win
        for pos in self.win_positions:
            if state[pos[0]] == state[pos[1]] == state[pos[2]] == player:
                return 100  # High reward for winning
        
        # Check for opponent win
        for pos in self.win_positions:
            if state[pos[0]] == state[pos[1]] == state[pos[2]] == opponent:
                return -100  # Severe penalty for losing
        
        # Draw
        if " " not in state:
            return 0
        
        # Strategic position evaluation
        reward = 0
        
        # Center control
        if state[4] == player:
            reward += 10
        elif state[4] == opponent:
            reward -= 10
        
        # Corner control
        corner_positions = [0, 2, 6, 8]
        for corner in corner_positions:
            if state[corner] == player:
                reward += 5
            elif state[corner] == opponent:
                reward -= 5
        
        # Potential win and blocking opportunities
        for pos in self.win_positions:
            player_count = sum(1 for i in pos if state[i] == player)
            opponent_count = sum(1 for i in pos if state[i] == opponent)
            empty_count = sum(1 for i in pos if state[i] == " ")
            
            # Reward for setting up a potential win
            if player_count == 2 and empty_count == 1:
                reward += 20
            
            # Penalty for opponent's potential win
            if opponent_count == 2 and empty_count == 1:
                reward -= 20
        
        return reward

    @staticmethod
    def is_terminal(state):
        win_positions = [(0, 1, 2), (3, 4, 5), (6, 7, 8), (0, 3, 6), (1, 4, 7), (2, 5, 8), (0, 4, 8), (2, 4, 6)]
        return any(state[pos[0]] == state[pos[1]] == state[pos[2]] != " " for pos in win_positions) or " " not in state

class PolicyIteration:
    def __init__(self, theta=0.01, discount_factor=0.9, max_iterations=1000):
        self.env = TicTacToeEnv()
        self.theta = theta
        self.discount_factor = discount_factor
        self.max_iterations = max_iterations
        
        # Initialize policy with random actions
        self.states = self.env.states
        self.V = {s: 0 for s in self.states}
        self.pi = {s: random.choice(self.env.actions(s)) if self.env.actions(s) else None for s in self.states}

    def policy_evaluation(self):
        """Evaluate the current policy"""
        for _ in range(self.max_iterations):
            delta = 0
            for s in self.states:
                if self.env.is_terminal(s):
                    self.V[s] = 0
                    continue

                if self.pi[s] is None:
                    continue

                v = self.V[s]
                
                # Compute value for the current policy
                next_state = self.env.transition_model(s, self.pi[s], "X")
                reward = self.env.evaluate_state(next_state, "X")
                
                self.V[s] = reward + self.discount_factor * self.V.get(next_state, 0)
                
                delta = max(delta, abs(v - self.V[s]))
            
            if delta < self.theta:
                break

    def policy_improvement(self):
        """Improve the policy based on current value function"""
        policy_stable = True
        
        for s in self.states:
            if self.env.is_terminal(s):
                continue

            old_action = self.pi[s]
            
            # Find the best action by looking at all possible actions
            action_values = []
            for a in self.env.actions(s):
                next_state = self.env.transition_model(s, a, "X")
                reward = self.env.evaluate_state(next_state, "X")
                action_value = reward + self.discount_factor * self.V.get(next_state, 0)
                action_values.append((action_value, a))
            
            # Select the action with the highest value
            if action_values:
                self.pi[s] = max(action_values, key=lambda x: x[0])[1]
            
            if old_action != self.pi[s]:
                policy_stable = False
        
        return policy_stable

    def policy_iteration(self):
        """Main policy iteration algorithm"""
        iterations = 0
        while iterations < self.max_iterations:
            # Evaluate current policy
            self.policy_evaluation()
            
            # Improve policy
            if self.policy_improvement():
                break
            
            iterations += 1

    def get_best_action(self, state):
        """Get the best action for a given state"""
        return self.pi.get(state, random.choice(self.env.actions(state)) if self.env.actions(state) else None)

    @staticmethod
    def print_board(state):
        """Print the current board state"""
        for i in range(0, 9, 3):
            print(state[i:i+3])
        print("\n")

    def play_game(self):
        """Play a game of Tic Tac Toe"""
        state = (" ",) * 9
        current_player = "X"

        while not self.env.is_terminal(state):
            self.print_board(state)

            if current_player == "X":
                while True:
                    try:
                        action = int(input("Enter your move (0-8): "))
                        if action in self.env.actions(state):
                            break
                        else:
                            print("Invalid move. Try again.")
                    except ValueError:
                        print("Please enter a number between 0-8.")
            else:
                action = self.get_best_action(state)

            state = self.env.transition_model(state, action, current_player)
            current_player = "O" if current_player == "X" else "X"

        self.print_board(state)

        # Determine winner
        for player in ["X", "O"]:
            if any(state[pos[0]] == state[pos[1]] == state[pos[2]] == player for pos in self.env.win_positions):
                print(f"Player {player} wins!")
                return

        print("It's a draw!")

def main():
    game = PolicyIteration()
    game.policy_iteration()
    game.play_game()

if __name__ == "__main__":
    main()

(' ', ' ', ' ')
(' ', ' ', ' ')
(' ', ' ', ' ')


('X', ' ', ' ')
(' ', ' ', ' ')
(' ', ' ', ' ')


('X', ' ', ' ')
(' ', ' ', ' ')
(' ', ' ', 'O')


('X', ' ', ' ')
(' ', 'X', ' ')
(' ', ' ', 'O')


('X', ' ', 'O')
(' ', 'X', ' ')
(' ', ' ', 'O')


('X', ' ', 'O')
(' ', 'X', 'X')
(' ', ' ', 'O')


('X', ' ', 'O')
(' ', 'X', 'X')
('O', ' ', 'O')


('X', 'X', 'O')
(' ', 'X', 'X')
('O', ' ', 'O')


('X', 'X', 'O')
('O', 'X', 'X')
('O', ' ', 'O')


('X', 'X', 'O')
('O', 'X', 'X')
('O', 'X', 'O')


Player X wins!


## First monte carlo visit 

In [9]:
import random
import math
import itertools

class TicTacToeEnv:
    def __init__(self):
        self.states = list(itertools.product(["X", "O", " "], repeat=9))
        self.win_positions = [
            (0, 1, 2), (3, 4, 5), (6, 7, 8),  # Rows
            (0, 3, 6), (1, 4, 7), (2, 5, 8),  # Columns
            (0, 4, 8), (2, 4, 6)  # Diagonals
        ]

    @staticmethod
    def actions(state):
        return [i for i, s in enumerate(state) if s == " "]

    @staticmethod
    def transition_model(state, action, player):
        return tuple(player if i == action else s for i, s in enumerate(state))

    def is_terminal(self, state):
        return any(
            state[pos[0]] == state[pos[1]] == state[pos[2]] != " " 
            for pos in self.win_positions
        ) or " " not in state

    def get_reward(self, state, player):
        opponent = "O" if player == "X" else "X"
        
        # Check for win
        for pos in self.win_positions:
            if state[pos[0]] == state[pos[1]] == state[pos[2]] == player:
                return 1  # Win
            if state[pos[0]] == state[pos[1]] == state[pos[2]] == opponent:
                return -1  # Loss
        
        # Draw
        if " " not in state:
            return 0
        
        return 0  # Ongoing game

class MonteCarloTreeSearch:
    def __init__(self, num_simulations=10000, exploration_weight=1.4):
        self.env = TicTacToeEnv()
        self.num_simulations = num_simulations
        self.exploration_weight = exploration_weight

    def monte_carlo_tree_search(self, initial_state):
        """Main MCTS algorithm"""
        class Node:
            def __init__(self, state, parent=None, action=None):
                self.state = state
                self.parent = parent
                self.action = action
                self.children = {}
                self.visits = 0
                self.value = 0
                self.untried_actions = TicTacToeEnv().actions(state)

            def fully_expanded(self):
                return len(self.untried_actions) == 0

            def best_child(self, c_puct):
                if not self.children:
                    return None
                
                return max(
                    self.children.values(), 
                    key=lambda child: child.value / (child.visits + 1e-6) + 
                        c_puct * math.sqrt(math.log(self.visits + 1) / (child.visits + 1e-6))
                )

        root = Node(initial_state)
        current_player = "X"

        for _ in range(self.num_simulations):
            node = root
            
            # Selection
            while not self.env.is_terminal(node.state) and node.fully_expanded():
                node = node.best_child(self.exploration_weight)
            
            # Expansion
            if not self.env.is_terminal(node.state):
                action = node.untried_actions.pop()
                next_state = self.env.transition_model(node.state, action, current_player)
                child_node = Node(next_state, parent=node, action=action)
                node.children[action] = child_node
                node = child_node

            # Simulation
            simulation_state = node.state
            sim_player = current_player
            while not self.env.is_terminal(simulation_state):
                actions = self.env.actions(simulation_state)
                action = random.choice(actions)
                simulation_state = self.env.transition_model(simulation_state, action, sim_player)
                sim_player = "O" if sim_player == "X" else "X"

            # Backpropagation
            result = self.env.get_reward(simulation_state, current_player)
            while node is not None:
                node.visits += 1
                node.value += result
                node = node.parent

        # Return the action with the most visits
        return max(root.children, key=lambda a: root.children[a].visits)

    def play_game(self):
        """Interactive game against MCTS AI"""
        state = (" ",) * 9
        current_player = "X"

        while not self.env.is_terminal(state):
            self.print_board(state)

            if current_player == "X":
                # AI's turn
                action = self.monte_carlo_tree_search(state)
            else:
                # Human's turn
                action = self.get_human_move(state)

            state = self.env.transition_model(state, action, current_player)
            current_player = "O" if current_player == "X" else "X"

        # Final board and result
        self.print_board(state)
        
        # Determine winner
        if any(state[pos[0]] == state[pos[1]] == state[pos[2]] == "X" for pos in self.env.win_positions):
            print("AI wins!")
        elif any(state[pos[0]] == state[pos[1]] == state[pos[2]] == "O" for pos in self.env.win_positions):
            print("You win!")
        else:
            print("It's a draw!")

    def get_human_move(self, state):
        """Get valid human move"""
        while True:
            try:
                action = int(input("Enter your move (0-8): "))
                if action in self.env.actions(state):
                    return action
                else:
                    print("Invalid move. Try again.")
            except ValueError:
                print("Please enter a number between 0-8.")

    def print_board(self, state):
        """Print the current board state"""
        for i in range(0, 9, 3):
            print(state[i:i+3])
        print("\n")

def main():
    mcts = MonteCarloTreeSearch(num_simulations=50000)
    mcts.play_game()

if __name__ == "__main__":
    main()

(' ', ' ', ' ')
(' ', ' ', ' ')
(' ', ' ', ' ')


(' ', ' ', ' ')
(' ', ' ', ' ')
(' ', 'X', ' ')


(' ', ' ', ' ')
(' ', 'O', ' ')
(' ', 'X', ' ')


('X', ' ', ' ')
(' ', 'O', ' ')
(' ', 'X', ' ')


('X', ' ', ' ')
('O', 'O', ' ')
(' ', 'X', ' ')


('X', ' ', ' ')
('O', 'O', ' ')
('X', 'X', ' ')


('X', 'O', ' ')
('O', 'O', ' ')
('X', 'X', ' ')


('X', 'O', ' ')
('O', 'O', ' ')
('X', 'X', 'X')


AI wins!
